<a href="https://colab.research.google.com/github/jimregan/wav2vec2-sprint/blob/comparison/Irish_comparisons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pocketsphinx

The pocketsphinx model doesn't have a real language model (it came from dictionary, so it's single words only, headwords only), so to show it in the best light, I'm using the data from the website for [Fuaimeanna na Gaeilge](http://www.fuaimeanna.ie/en/) *("The Sounds of Irish")*, which has equivalent pronunciation examples. I've put an old scraper for the site [here](https://github.com/jimregan/wav2vec2-sprint/blob/main/irish/fuaimeanna.pl); this writes a .tsv file with the data, and a shell script to use wget to download the sounds. I already have the data, so I'm just uploading it, but during the sprint I wrote a [script](https://github.com/jimregan/wav2vec2-sprint/blob/main/irish/convert-fuaimeanna-csv.pl) to convert the .tsv to a .csv that `datasets` could read more easily.

Setting up is easy:

In [ ]:
!apt-get install pocketsphinx

Next, grab the pretrained model:

In [ ]:
!wget https://github.com/jimregan/irish-asr-data/releases/download/teanglann-0.1/cmusphinx-ga-teanglann-0.1.zip

In [ ]:
!unzip cmusphinx-ga-teanglann-0.1.zip

In [ ]:
!unzip fuaimeanna.zip

Pocketsphinx comes from the bad old days before audio libraries were something that could be relied on being present, so the files need to be 16k `.wav`

In [ ]:
!for i in fuaimeanna/mp3/*.mp3;do ffmpeg -i "$i" -acodec pcm_s16le -ac 1 -ar 16000 "$i.wav";done

In [ ]:
!for i in fuaimeanna/mp3/*.wav; do f=$(echo $i|awk -F/ '{print $NF}');printf "%s\t" $f >> ps-output; pocketsphinx_continuous -infile $i -hmm cmusphinx-ga-teanglann-0.1/ -dict cmusphinx-ga-teanglann-0.1/ga.dic -lm cmusphinx-ga-teanglann-0.1/ga.lm.DMP >> ps-output;done

In [ ]:
!pip install jiwer

In [33]:
import csv
data = dict()
with open("/content/fuaimeanna/all-fuaimeanna-data.tsv") as file:
    all = csv.reader(file, delimiter="\t", quotechar=None)
    for row in all:
      if row[0] == 'Orthographic':
        continue
      else:
        file1 = row[1].replace('/sounds/', '')
        data[file1] = row[0]
        file2 = row[3].replace('/sounds/', '')
        data[file2] = row[0]
        file3 = row[5].replace('/sounds/', '')
        data[file3] = row[0]
merged = list()
with open("ps-output") as file:
    ps = csv.reader(file, delimiter="\t", quotechar=None)
    for row in ps:
      if len(row) != 2:
        continue
      filename = row[0].replace('.wav', '')
      add=(row[1],data[filename])
      merged.append(add)
lista = [a[0] for a in merged]
listb = [a[1] for a in merged]

In [42]:
from jiwer import wer
result = wer(lista, listb)
'{:.2f}'.format(result)

'0.99'

In [ ]:
!pip install deepspeech

In [ ]:
!wget https://github.com/jimregan/DeepSpeech/releases/download/0.8.2-ga-test/output_graph_ga.pbmm https://github.com/jimregan/DeepSpeech/releases/download/0.8.2-ga-test/kenlm.scorer

In [ ]:
!for i in fuaimeanna/mp3/*.wav;do f=$(echo $i|awk -F/ '{print $NF}'); printf "%s\t" $f >> ds-output; deepspeech --model output_graph_ga.pbmm --scorer kenlm.scorer --audio $i >> ds-output;done

Loading model from file output_graph_ga.pbmm
TensorFlow: v2.3.0-6-g23ad988
DeepSpeech: v0.9.3-0-gf2e9c85
2021-03-30 01:19:41.980298: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Loaded model in 0.0126s.
Loading scorer from files kenlm.scorer
Loaded scorer in 0.000268s.
Running inference.
Inference took 1.391s for 1.907s audio file.
Loading model from file output_graph_ga.pbmm
TensorFlow: v2.3.0-6-g23ad988
DeepSpeech: v0.9.3-0-gf2e9c85
2021-03-30 01:19:43.621273: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild 